In [3]:
import os
# 【硬性要求】加载环境变量
from dotenv import load_dotenv
load_dotenv(override=True)

from langchain_community.document_loaders import PyPDFLoader
# 导入递归字符文本分割器
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ==========================================
# 1. 准备数据源 (承接上一节课的流程)
# ==========================================
# 假设我们有一个名为 handbook.pdf 的文件
PDF_PATH = "data/handbook.pdf"

# 占位逻辑：如果本地没有文件，生成一个测试用例以保证代码可运行
if not os.path.exists(PDF_PATH):
    os.makedirs("data", exist_ok=True)
    with open(PDF_PATH, "w") as f:
        # 我们不用 FPDF 生成真实 PDF 也可以，这里仅为示意，
        # 在真实环境中请确保 handbook.pdf 存在。
        pass 

try:
    loader = PyPDFLoader(PDF_PATH)
    # 按页物理分割，返回 Document 列表
    pages = loader.load_and_split() 
    print(f"✅ 成功加载 PDF，共计 {len(pages)} 页。")

    # ==========================================
    # 2. 实施分块策略 (Chunking Strategy)
    # ==========================================
    # 实例化文本分割器
    # chunk_size=200: 每块尽量包含 200 个字符
    # chunk_overlap=50: 相邻块之间保留 50 个字符的重叠冗余
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200, 
        chunk_overlap=120
    )
    
    # 传入页面列表，执行深入分割
    chunks = text_splitter.split_documents(pages)
    
    # 验证分块结果，例如 3 页的 PDF 被切分成了数十个小块
    print(f"🔪 分块完成！原始页面被切分成了 {len(chunks)} 个独立的 Chunk。")

    # ==========================================
    # 3. 验证重叠与元数据保留 (科研重点观察)
    # ==========================================
    if len(chunks) >= 2:
        print("\n=== 观察 Chunk 0 ===")
        print(f"内容: {chunks[0].page_content}")
        # 观察元数据是否完好传递（用于后续溯源引用）
        print(f"元数据: {chunks[0].metadata}") 
        
        print("\n=== 观察 Chunk 1 ===")
        print(f"内容: {chunks[1].page_content}")
        print(f"元数据: {chunks[1].metadata}")
        
        print("\n💡 【私教提示】对比 Chunk 0 的结尾和 Chunk 1 的开头。")
        print("你会发现有一段重复的话 (Overlap)，这正是为了保证检索时语义不被生硬切断！")

        # ==========================================
        # 4. Overlap 自检（量化验证，避免肉眼误判）
        # ==========================================
        def _max_overlap(a: str, b: str, max_len: int = 200) -> int:
            """返回 a 的后缀与 b 的前缀的最大完全重叠长度（字符数）。"""
            best = 0
            limit = min(len(a), len(b), max_len)
            for n in range(1, limit + 1):
                if a[-n:] == b[:n]:
                    best = n
            return best

        tail = chunks[0].page_content[-120:]
        head = chunks[1].page_content[:120]
        print("\n=== Overlap 自检 ===")
        print(f"Chunk 0 尾部(120 chars): {repr(tail)}")
        print(f"Chunk 1 头部(120 chars): {repr(head)}")
        overlap_len = _max_overlap(chunks[0].page_content, chunks[1].page_content)
        print(f"✅ 检测到的实际 overlap 长度: {overlap_len} 字符")

        if overlap_len == 0:
            print("\n说明：你这里看不到 overlap 很常见，原因通常是 PDF 文本里有硬换行(\\n)或段落切分导致最小切分片段本身就 > chunk_overlap(50)。")
            print("RecursiveCharacterTextSplitter 的 overlap 是按‘片段’回退保留的，无法保留片段的一部分，所以可能出现 overlap_len=0。")
            print("可选修复：把 chunk_overlap 调大（例如 120），或先将 page_content 中的 \\n 替换为空格再进行分块。\n")

except Exception as e:
    print(f"执行异常，请确保 '{PDF_PATH}' 是一个有效的 PDF 文件。错误信息: {e}")

✅ 成功加载 PDF，共计 3 页。
🔪 分块完成！原始页面被切分成了 81 个独立的 Chunk。

=== 观察 Chunk 0 ===
内容: In the contemporary digital landscape, the "datasphere" is expanding at an
unprecedented rate. This explosion of connectivity has fundamentally altered the
元数据: {'producer': '', 'creator': 'WPS 文字', 'creationdate': '2026-02-24T21:43:41-05:00', 'author': '', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2026-02-24T21:43:41-05:00', 'sourcemodified': "D:20260224214341-05'00'", 'subject': '', 'title': '', 'trapped': '/False', 'source': 'data/handbook.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}

=== 观察 Chunk 1 ===
内容: unprecedented rate. This explosion of connectivity has fundamentally altered the
social contract between individuals and organizations. Every interaction, from a
元数据: {'producer': '', 'creator': 'WPS 文字', 'creationdate': '2026-02-24T21:43:41-05:00', 'author': '', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2026-02-24T21:43:41-05:00', 'sourcemodified': "D:20260224214